In [38]:
import pandas as pd
import re
import spacy
import plac
from pathlib import Path
from spacy.util import minibatch, compounding
from __future__ import unicode_literals, print_function
import csv

import src.funconstructor as fc
import src.train as train

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import savefig
import pickle
import random

In [8]:
df = pd.read_csv('input/#jokermovie.csv',)

In [9]:
categories = {
    'CHARACTERS': ['character', 'characters', 'Arthur Fleck', 'Murray Franklin', 'Sophie Dumond', 'Penny Fleck', 'mother', 'mum'],
    'TALENT': ['actor', 'actress', 'actors', 'talent', 'director', 'Joaquin Phoenix', '#JoaquinPhoenix', 'Robert De Niro', 'Zazie Beetz', 'Frances Conroy', 'Todd Philips'],
    'PLOT': ['plot', 'frame', 'scene', 'scenes'],
    'TICKETS': ['ticket', 'tickets', 'presale'],
    'SOUNDTRACK': ['music', 'song'],
    'SPECIAL EFFECTS': ['FX', 'visual effects', 'graphics', 'postproduction'],
    'COSTUME DESIGN': ['costum', 'costumes', 'costume designer'],
    'HYPE': ['hype', 'hyyyype', 'amazing', 'awesome', 'desire', 'desires', 'premiere', 'Cant wait'],
    'BOX OFFICE': ['box office'],
    'REVIEWS': ['review', 'reviews'],
    'TRAILER':['trailer', 'trailers']
}

In [ ]:
main(training_data,output_dir='output/model')

In [10]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(TRAIN_DATA, model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [11]:
training_data = train.trainData(df, categories)

Created blank 'en' model
Losses {'ner': 2267.009393120841}
Losses {'ner': 677.989675453814}
Losses {'ner': 560.0222158168289}
Losses {'ner': 522.8560353791358}
Losses {'ner': 454.70788200820544}
Losses {'ner': 416.38821700176646}
Losses {'ner': 366.9021494486867}
Losses {'ner': 317.44688021366824}
Losses {'ner': 373.6888354530711}
Losses {'ner': 325.03338679433114}
Losses {'ner': 264.27272987658034}
Losses {'ner': 252.73611809014648}
Losses {'ner': 250.1228712142907}
Losses {'ner': 261.95623314044826}
Losses {'ner': 240.402294421152}
Losses {'ner': 227.54305814485394}
Losses {'ner': 230.54185107877476}
Losses {'ner': 200.3347444435599}
Losses {'ner': 217.60408266101126}
Losses {'ner': 203.82556376000792}
Losses {'ner': 194.2419272149259}
Losses {'ner': 206.83845494290114}
Losses {'ner': 149.08452869497953}
Losses {'ner': 219.29900407491405}
Losses {'ner': 162.73712770798423}
Losses {'ner': 179.01020559253666}
Losses {'ner': 206.1505965320396}
Losses {'ner': 136.57118877595366}
Losses {

Entities []
Tokens [('Military', '', 2), ('commanders', '', 2), ('are', '', 2), ('warning', '', 2), ('service', '', 2), ('members', '', 2), ('about', '', 2), ('the', '', 2), ('potential', '', 2), ('for', '', 2), ('mass', '', 2), ('shootings', '', 2), ('at', '', 2), ('screenings', '', 2), ('of', '', 2), ('the', '', 2), ('upcoming', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('according', '', 2), ('to', '', 2), ('media', '', 2), ('reports', '', 2), ('.', '', 2), ('\n', '', 2), ('https://bit.ly/2nlpvBd', '', 2), ('\xa0', '', 2)]
Entities []
Tokens [('Joker', '', 2), ('week', '', 2), ('starts', '', 2), ('todau', '', 2), ('.', '', 2), ('\n\n', '', 2), ('#', '', 2), ('Joker', '', 2), ('\n', '', 2), ('#', '', 2), ('jokermovie', '', 2), ('https://www.instagram.com/p/B3DdgALgSz_/?igshid=1wu9ie7oz3dsa', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('Ca', '', 2), ("n't", '', 2), ('wait', '', 2), ('to', '', 2), ('see', '', 2), ('this', '', 2), ('masterpiece', '', 2), ('.', '', 2), 

Entities []
Tokens [('Tickets', '', 2), ('to', '', 2), ('see', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('have', '', 2), ('been', '', 2), ('booked', '', 2), ('!', '', 2), ('!', '', 2)]
Entities []
Tokens [('All', '', 2), ('the', '', 2), ('big', '', 2), ('fearmongering', '', 2), ('babies', '', 2), ('cried', '', 2), ('over', '', 2), ('nothing', '', 2), ('.', '', 2), ('\n\n', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('#', '', 2), ('jokerpic.twitter.com/4zev55s6D4', '', 2)]
Entities []
Tokens [('Ca', '', 2), ("n't", '', 2), ('wait', '', 2), ('to', '', 2), ('see', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('tonight', '', 2), ('!', '', 2), ('!', '', 2), ('!', '', 2), ('4', '', 2), ('minute', '', 2), ('countdown', '', 2), ('.', '', 2)]
Entities []
Tokens [('“', '', 2), ('Why', '', 2), ('So', '', 2), ('Serious', '', 2), ('?', '', 2), ('!', '', 2), ('”', '', 2), ('\n', '', 2), ('Harley', '', 2), ('and', '', 2), ('Joker', '', 2), ('from', '', 2), ('INJUSTICE', '', 2), (',', '', 2), ('

Entities []
Tokens [('So', '', 2), ('excited', '', 2), ('to', '', 2), ('see', '', 2), ('@jokermovie', '', 2), ('!', '', 2), ('@showbizshelly', '', 2), ('is', '', 2), ('the', '', 2), ('🔌', '', 2), ('🔌', '', 2), ('🔌', '', 2), ('\n', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('pic.twitter.com/8aXkh6HWMd', '', 2)]
Entities []
Tokens [('HAHAHAHAHAHAHA', '', 2), ('\n', '', 2), ('🦇', '', 2), ('😳', '', 2), ('\n', '', 2), ('@IGN', '', 2), ('@nerdist', '', 2), ('@CBR', '', 2), ('@ComicBook', '', 2), ('@ComicBookNOW', '', 2), ('\n', '', 2), ('#', '', 2), ('batman', '', 2), ('#', '', 2), ('batmanandrobin', '', 2), ('#', '', 2), ('jasontodd', '', 2), ('#', '', 2), ('joker', '', 2), ('#', '', 2), ('harleyquinn', '', 2), ('#', '', 2), ('jokerfilm', '', 2), ('#', '', 2), ('jokermovie', '', 2), ('#', '', 2), ('jokeredit', '', 2), ('#', '', 2), ('jokerandharleyquinn', '', 2), ('#', '', 2), ('redhoodfanseries', '', 2), ('#', '', 2), ('redhood', '', 2), ('#', '', 2), ('filmmaking', '', 2), ('#', '', 2)

Entities []
Tokens [('Oh', '', 2), ('I', '', 2), ('’m', '', 2), ('super', '', 2), ('excited', '', 2), ('.', '', 2), ('  ', '', 2), ('Just', '', 2), ('one', '', 2), ('day', '', 2), ('to', '', 2), ('go', '', 2), ('!', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('pic.twitter.com/P7xqBMuvG6', '', 2)]
Entities []
Tokens [('Just', '', 2), ('a', '', 2), ('few', '', 2), ('more', '', 2), ('days', '', 2), ('#', '', 2), ('countdown', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('pic.twitter.com/sBlsU6hAI4', '', 2)]
Entities [('#JoaquinPhoenix', 'TALENT')]
Tokens [('Wow', '', 2), ('.', '', 2), ('A', '', 2), ('flawless', '', 2), (',', '', 2), ('breathtaking', '', 2), (',', '', 2), ('tragic', '', 2), (',', '', 2), ('funny', '', 2), ('and', '', 2), ('disturbing', '', 2), ('performance', '', 2), ('by', '', 2), ('#', 'TALENT', 3), ('JoaquinPhoenix', 'TALENT', 1), ('in', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('.', '', 2), ('Incredible', '', 2), ('.', '', 2), ('🤡

...

Entities [('song', 'SOUNDTRACK')]
Tokens [('U', '', 2), ('all', '', 2), ('know', '', 2), ('&', '', 2), ('probably', '', 2), ('even', '', 2), ('love', '', 2), ('"', '', 2), ('the', '', 2), ('hey', '', 2), ('song', 'SOUNDTRACK', 3), ('"', '', 2), ('without', '', 2), ('even', '', 2), ('realising', '', 2), ('it', '', 2), ('.', '', 2), ('Gary', '', 2), ('Glitter', '', 2), ('created', '', 2), ('it', '', 2), ('&', '', 2), ('everytime', '', 2), ('u', '', 2), ('listen', '', 2), ('to', '', 2), ('it', '', 2), (',', '', 2), ('u', '', 2), ('are', '', 2), ('ensuring', '', 2), ('that', '', 2), ('a', '', 2), ('convicted', '', 2), ('child', '', 2), ('rapist', '', 2), ('gets', '', 2), ('a', '', 2), ('royalty', '', 2), ('paycheck', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('BoycottJoker', '', 2), ('#', '', 2), ('ToddPhillipshttps://www.billboard.com/articles/business/6273991/gary-glitters-hey-song-stadium-still-played', '', 2), ('\xa0', '', 

Entities []
Tokens [('imo', '', 2), ('film', '', 2), ('of', '', 2), ('the', '', 2), ('year', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('https://twitter.com/BatmanNewsCom/status/1179894583267741698', '', 2), ('\xa0', '', 2), ('…', '', 2)]
Entities []
Tokens [('Who', '', 2), ('love', '', 2), ('Joker', '', 2), ('🤡', '', 2), ('?', '', 2), ('YES', '', 2), ('!', '', 2), ('!', '', 2), ('!', '', 2), ('Joker', '', 2), ('Come', '', 2), ('back', '', 2), ('again', '', 2), ('🤡', '', 2), ('\n', '', 2), ('#', '', 2), ('ToddPhillips', '', 2), ('\n', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('#', '', 2), ('JokerOCT4', '', 2), ('#', '', 2), ('HHN29VIDEO', '', 2), ('#', '', 2), ('HHN29PHOTO', '', 2), ('#', '', 2), ('HalloweenOCTjoker', '', 2), ('#', '', 2), ('JokerFanpic.twitter.com', '', 2), ('/', '', 2), ('oKSrKhwsh7', '', 2)]
Entities [('#JoaquinPhoenix', 'TALENT')]
Tokens [('@jokermovie', '', 2), ('kills', '', 2), ('it', '', 2), ('!', '', 2), ('My', '', 2), ('photos', '', 2), ('

Entities []
Tokens [('Tonights', '', 2), ('Topics', '', 2), ('on', '', 2), ('#', '', 2), ('TheBarbeque', '', 2), ('\n\n', '', 2), ('-', '', 2), ('R.', '', 2), ('Kelly', '', 2), ('asks', '', 2), ('for', '', 2), ('bail', '', 2), ('\n', '', 2), ('-', '', 2), ('James', '', 2), ('Franco', '', 2), ('’s', '', 2), ('odd', '', 2), ('ways', '', 2), ('\n', '', 2), ('-', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('security', '', 2), ('\n', '', 2), ('-', '', 2), ('Summer', '', 2), ('Walker', '', 2), ('dropping', '', 2), ('an', '', 2), ('album', '', 2), ('!', '', 2), ('\n\n', '', 2), ('+', '', 2), ('More', '', 2), ('!', '', 2), ('Tune', '', 2), ('in', '', 2), ('NOW', '', 2), ('!', '', 2), ('🥩', '', 2), ('🔥', '', 2)]
Entities []
Tokens [('Joker', '', 2), ('Poster', '', 2), ('Design', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('#', '', 2), ('Poster', '', 2), ('#', '', 2), ('DCComicspic.twitter.com', '', 2), ('/', '', 2), ('kO3VlurMLj', '', 2)]
Entities []
Tokens [('All', '', 2), ('joking', '', 2

Entities [('Joaquin Phoenix', 'TALENT')]
Tokens [('Despite', '', 2), ('an', '', 2), ('incredible', '', 2), ('performance', '', 2), ('from', '', 2), ('Joaquin', 'TALENT', 3), ('Phoenix', 'TALENT', 1), (',', '', 2), ('@mistermainevent', '', 2), ('did', '', 2), ('n’t', '', 2), ('much', '', 2), ('care', '', 2), ('for', '', 2), ('JOKER', '', 2), ('.', '', 2), ('Full', '', 2), ('review', '', 2), ('here', '', 2), (':', '', 2), ('https://www.lamplightreview.com/joker-movie-review-joaquin-phoenix/', '', 2), ('\xa0', '', 2), ('…', '', 2), ('#', '', 2), ('Joker', '', 2), ('#', '', 2), ('JokerMovie', '', 2)]
Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('opens', '', 2), ('in', '', 2), ('2', '', 2), ('days', '', 2), ('.', '', 2), ('We', '', 2), ('are', '', 2), ('counting', '', 2), ('down', '', 2), ('the', '', 2), ('top', '', 2), ('5', '', 2), ('actors', '', 2), ('to', '', 2), ('portray', '', 2), ('the', '', 2), ('Clown', '', 2), ('Prince', '', 2), ('of', '', 2), ('Crime', '', 2), ('.', 

Entities [('character', 'CHARACTERS')]
Tokens [('I', '', 2), ('do', '', 2), ('also', '', 2), ('think', '', 2), ('you', '', 2), ('need', '', 2), ('to', '', 2), ('be', '', 2), ('a', '', 2), ('fan', '', 2), ('of', '', 2), ('the', '', 2), ('character', 'CHARACTERS', 3), ('himself', '', 2), ('to', '', 2), ('actually', '', 2), ('appreciate', '', 2), ('the', '', 2), ('angle', '', 2), ('of', '', 2), ('where', '', 2), ('the', '', 2), ('film', '', 2), ('was', '', 2), ('coming', '', 2), ('from', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2)]
Entities [('trailer', 'TRAILER')]
Tokens [('Repost', '', 2), ('from', '', 2), ('#', '', 2), ('ToddPhillips', '', 2), (':', '', 2), ('Teaser', '', 2), ('trailer', 'TRAILER', 3), ('tomorrow', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('–', '', 2), ('in', '', 2), ('theaters', '', 2), ('October', '', 2), ('4', '', 2), ('.', '', 2), ('\n\n', '', 2), ('https://twitter.com/jokermovie/status/1113154655402336256', '', 2), ('\xa0', '', 2), 

In [17]:
def categoryColumn(path, df):
    #Predict category while iterate through text serie and save the final DataFrame 
    nlp = spacy.load(path)

    for index, row in df.iterrows():
        doc = nlp(row['text'])
        for ent in doc.ents:
            df.loc[index, 'category'] = [(ent.label_)]
    
    return df.to_csv('output/predicted.csv')

In [18]:
categoryColumn('output/model', df) #genera predicted.csv

In [20]:
dfp = pd.read_csv('output/predicted.csv')
dfp.head()

,Unnamed: 0,id,author_id,author_name,author_handler,author_avatar,lang,type,text,date,...,links,links_first,image_links,image_links_first,rp_user_id,rp_user_name,location,tweet_link,search,category
0,0,1180633752885743616,2866588387,BagMan,The_BagMan_,https://pbs.twimg.com/profile_images/123315125...,en,original,People at the cinema thinking they got the #Jo...,2019-10-05T23:59:56.000Z,...,"[""https://t.co/FULbcu3SBj""]",https://t.co/FULbcu3SBj,[],NaN,NaN,NaN,NaN,https://twitter.com/The_BagMan_/status/1180633...,#jokermovie,PLOT
1,1,1180633751744913408,77537588,david,bizzarr1000,https://pbs.twimg.com/profile_images/122677952...,en,original,I have a new favorite joker @jokermovie #Joker...,2019-10-05T23:59:56.000Z,...,"[""https://t.co/S4wOE3lmQF""]",https://t.co/S4wOE3lmQF,[],NaN,NaN,NaN,NaN,https://twitter.com/bizzarr1000/status/1180633...,#jokermovie,NaN
2,2,1180633723932483584,701525750,Ijaz Moin,ijjy7,https://pbs.twimg.com/profile_images/866198526...,en,original,"⭐⭐⭐⭐⭐ Performance, what a movie #JokerMovie pi...",2019-10-05T23:59:49.000Z,...,"[""https://t.co/vGun5i7WPW""]",https://t.co/vGun5i7WPW,"[""https://pbs.twimg.com/media/EGJ0HV3WwAYkdgQ....",https://pbs.twimg.com/media/EGJ0HV3WwAYkdgQ.jpg,NaN,NaN,NaN,https://twitter.com/ijjy7/status/1180633723932...,#jokermovie,NaN
3,3,1180633642198077441,437109027,Brandon Denson,BrandonDeShane2,https://pbs.twimg.com/profile_images/119195475...,en,original,#JokerMovie was absolutely amazing and ended w...,2019-10-05T23:59:30.000Z,...,"[""https://t.co/idTvidHpbE""]",https://t.co/idTvidHpbE,[],NaN,NaN,NaN,NaN,https://twitter.com/BrandonDeShane2/status/118...,#jokermovie,HYPE
4,4,1180633577693880321,1644317684,Adam,Klakuladd,https://pbs.twimg.com/profile_images/123146325...,en,original,I just want to give both Arthur Fleck and Gary...,2019-10-05T23:59:14.000Z,...,[],NaN,[],NaN,NaN,NaN,NaN,https://twitter.com/Klakuladd/status/118063357...,#jokermovie,CHARACTERS


In [39]:
def dataReady(pathcsv,pathcategories):
    #From csv to id and text series DataFrame
    df = pd.read_csv(pathcsv)
    df = df[['id', 'text']]

    #From pkl file to categories dictionary
    categories = fc.pkltodict(pathcategories)

    #Trained model 
    training_data = train.trainData(df, categories)

    #Predicted df to csv
    dffinal = categoryColumn('output/model', df) 

    #Graph
    dffinal = pd.read_csv('predicted.csv') 
    graph = sns.countplot(data=dffinal, x = 'category')
    plt.xticks(rotation=45)
    plt.savefig('../output/barcategories') 

    #PDF 
    #incluir la función que llame a createpdf

    print('It will take a few minutes. You will have your report PDF in the output folder') 

In [ ]:
dataReady('input/#jokermovie.csv', '../data.csv')

Created blank 'en' model
Losses {'ner': 2205.969161499701}
Losses {'ner': 682.0164436046074}
Losses {'ner': 574.5397997295139}
Losses {'ner': 516.1722827262553}
Losses {'ner': 446.5483549459477}
Losses {'ner': 392.5611025377506}
Losses {'ner': 378.60750416498996}
Losses {'ner': 327.7727282084941}
Losses {'ner': 328.4521792744106}
Losses {'ner': 306.6854476190137}
Losses {'ner': 316.08843124681414}
Losses {'ner': 280.3439097283851}
Losses {'ner': 247.61022297750114}
Losses {'ner': 273.99951339249077}
Losses {'ner': 285.10850326759925}
Losses {'ner': 225.01085500845437}
Losses {'ner': 219.95193473619983}
Losses {'ner': 237.45524490793798}
Losses {'ner': 229.92885009059964}
Losses {'ner': 211.75178057760957}
Losses {'ner': 232.54804114557166}
Losses {'ner': 190.70258741062688}
Losses {'ner': 166.9641141429721}
Losses {'ner': 201.36529102372046}
Losses {'ner': 176.49734058829438}
Losses {'ner': 146.5848164677376}
Losses {'ner': 201.3786898344536}
Losses {'ner': 167.6858826117215}
Losses {'